### 출력파서의 역할
- LLM의 출력을 받아 더 적합한 형식으로 변환
- 구조화된 데이터 생성에 매우 유용
- LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith('ch3_pydantic',set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
ch3_pydantic


In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

llm = ChatOpenAI(model_name = "gpt-4o-mini")

In [13]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


In [14]:
from langchain_teddynote.messages import stream_response
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

중요한 내용 요약:

- 발신자: 김철수 (바이크코퍼레이션 상무)
- 수신자: 이은채 (Teddy International)
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 요청 사항: ZENESIS 모델의 상세 브로슈어(기술 사양, 배터리 성능, 디자인 정보)
- 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시에 귀사 사무실에서 만나고자 함.

In [15]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [16]:

# PydanticOutputParser의 format_instructions 가져오기
format_instructions = parser.get_format_instructions()

In [17]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# Prompt에 부분 포맷팅 적용
prompt = prompt.partial(format=format_instructions)


In [18]:
chain = prompt|llm
response = chain.stream(
    {
        "email_conversation":email_conversation,
        'question':"이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

output = stream_response(response,return_output= True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수가 이은채 대리에게 ZENESIS 자전거에 대한 브로슈어 요청과 유통 전략 협의 미팅을 제안함.",
  "date": "1월 15일 오전 10시"
}
```

In [19]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)


person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='바이크코퍼레이션의 김철수가 이은채 대리에게 ZENESIS 자전거에 대한 브로슈어 요청과 유통 전략 협의 미팅을 제안함.' date='1월 15일 오전 10시'


In [21]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser
response = chain.invoke(
    {
        "email_conversation":email_conversation,
        'question':"이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

response


EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 이은채 대리에게 ZENESIS 자전거에 대한 브로슈어 요청과 유통 전략 및 마케팅 계획 논의를 위한 미팅을 제안함.', date='1월 15일 오전 10시')

### 튜토리얼 내용과 pydantic 버전이 맞지 않아서 실행이 안되는 듯 -> 그냥 v1 안 쓰면 됨